In [1]:
# Carga de librerias
import numpy as np 
import pandas as pd 
import os
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import glob
import os
from PIL import Image
from itertools import chain
from collections import Counter
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.preprocessing.image import DirectoryIterator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import accuracy_score
from keras import backend as K
from itertools import chain
# Definición de las variables globales
IMG_SIZE=32
NUM_CHANNELS=3
NUM_CLASSES=43


In [3]:
# Declaración de los directorios
current_dir=os.getcwd()
dataset_dir=current_dir+"/Dataset"
dir_train=dataset_dir+"/Train"
train_data=pd.read_csv(dataset_dir+"/Train.csv")
test_data=pd.read_csv(dataset_dir+"/Test.csv")
meta_data=pd.read_csv(dataset_dir+"/Meta.csv")

x_images=test_data["Path"]
x_train_images=train_data["Path"]
x_train_labels=train_data[['ClassId']].to_numpy()
y_test=test_data[['ClassId']].values



In [4]:
# Generacion de los datos de entrenamiento
def generate_train_data(IMG_SIZE, x_train_images,x_labels,brig):
    data=[]
    labels=x_labels
    for i in x_train_images:
        try:
            image=cv2.imread(dataset_dir+"/"+i)
            image=change_brightness(image,brig)
            image_from_array = Image.fromarray(image, 'RGB')
            size_image = image_from_array.resize((IMG_SIZE, IMG_SIZE))
            data.append(np.array(size_image))
        except AttributeError:
            print("Error")
    data = np.array(data)
    labels = np.array(labels.ravel())
    shuffle_indexes = np.arange(data.shape[0])
    np.random.shuffle(shuffle_indexes)
    data = data[shuffle_indexes]
    labels = labels[shuffle_indexes]
    X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42, shuffle=True)

    X_train = X_train/255 
    X_val = X_val/255

    y_train = to_categorical(y_train, NUM_CLASSES)
    y_val = to_categorical(y_val, NUM_CLASSES)

    return X_train, X_val, y_train,y_val  


In [5]:
# Función que cambia el brillo
def change_brightness(img, value):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    v = cv2.add(v,value)
    v[v > 255] = 255
    v[v < 0] = 0
    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

In [6]:
# Generacion datos de test
def test_data(test_data, IMG_SIZE,brig):
    data=[]
    for i in test_data:
        try:
            image=cv2.imread(dataset_dir+"/"+i)
            image=change_brightness(image,brig)
            image_from_array = Image.fromarray(image, 'RGB')
            size_image = image_from_array.resize((IMG_SIZE, IMG_SIZE))
            data.append(np.array(size_image))
        except AttributeError:
            print("Error")
    X_test=np.array(data)
    X_test=X_test.astype('float32')/255
    return X_test

In [7]:
# Configuracion de los modelos CNN
settings_trained_models = {
    'cnn_92': {
        'conv': [64,64,64],
        'dens': [256,256],
        'name': "cnn_92"
    },
    'cnn_74': {
        'conv': [16,64,64],
        'dens': [128,128],
        'name': "cnn_74"
    },
    'cnn_17': {
        'conv': [32,64,64],
        'dens': [],
        'name': "cnn_17"
    },
    'cnn_82': {
        'conv': [32,32,64],
        'dens': [256,256],
        'name': "cnn_82"
    },

    'cnn_54': {
        'conv': [16,16,32],
        'dens': [128,128],
        'name': "cnn_54"
    },
    'cnn_67': {
        'conv': [16,32,64],
        'dens': [256,256],
        'name': "cnn_67"
    },
    'cnn_91': {
        'conv': [64,64,64],
        'dens': [128],
        'name': "cnn_91"
    },
    'cnn_61': {
        'conv': [16,32,32],
        'dens': [128],
        'name': "cnn_61"
    },
    'cnn_66': {
        'conv': [16,32,64],
        'dens': [128],
        'name': "cnn_66"
    },
    'cnn_84': {
        'conv': [32,32,64],
        'dens': [128,128],
        'name': "cnn_84"
    },
       
}

In [8]:
# Reentrenamiento de los modelos
def retrain_model(settings,X_train,X_val,y_train,y_val,X_test,y_test, brig):
    history_results=[]
    results=[]
    for name, model_settings in settings.items():
        h5_model=current_dir+'/Results/CNN_Train/'+name+'.h5'
        model=tf.keras.models.load_model(h5_model)
        epochs = 20
        history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_val, y_val))
        history_results.append((history,name))
        pred=np.argmax(model.predict(X_test), axis=-1)
        val_acc=accuracy_score(y_test,pred)
        result={"Model": name+"_"+str(brig),
          "Accuracy": val_acc
            }
        results.append(result)
        
    results_table=pd.DataFrame(results)
    return history_results, results_table
      #return model, model_name, history


In [9]:
# PLoteo del progreso de la precision
def plot_history(history_model,brig):
    list_models=[]
    fig, (ax1) = plt.subplots(1,1, figsize=(12, 10))
    for i,(h,model_name) in enumerate(history_model):
   # Visualizamos la evolución de la accuracy
        ax1.plot(h.history['val_accuracy'])
        ax1.set_title("Brightness factor "+ str(brig))
        ax1.set_ylabel('Accuracy')
        ax1.set_xlabel('Epoch')
        list_models.append(str(model_name))
    fig.legend(list_models, loc="right")
# Visualizamos la evolución del error cometido por la red
    plt.show()


In [ ]:
# Ataque factor -80
X_train,X_val,y_train,y_val=generate_train_data(IMG_SIZE, x_train_images,x_train_labels,-80)
X_test=test_data(x_images, IMG_SIZE,-80)
history_80,result_80=retrain_model(settings_trained_models,X_train,X_val,y_train,y_val,X_test,y_test,-80)
plot_history(history_80,-80)
result_80.to_csv(current_dir+'/Results/CNN_Defense/CNN_10_retrained_80.csv')

# Ataque factor 180
X_train,X_val,y_train,y_val=generate_train_data(IMG_SIZE, x_train_images,x_train_labels,180)
X_test=test_data(x_images, IMG_SIZE,180)
history_180,result_180=retrain_model(settings_trained_models,X_train,X_val,y_train,y_val,X_test,y_test,180)
plot_history(history_180,180)
result_180.to_csv(current_dir+'/Results/CNN_Defense/CNN_10_retrained_180.csv')




In [ ]:
# Ataque factores -80 y 180
train_1, train_2, label_1, label_2 = train_test_split(x_train_images, x_train_labels, test_size=0.5, random_state=42, shuffle=True)

X_train_1,X_val_1,y_train_1,y_val_1=generate_train_data(IMG_SIZE, train_1,label_1,180)
X_train_2,X_val_2,y_train_2,y_val_2=generate_train_data(IMG_SIZE, train_2,label_2,-80)

X_train_combined=np.concatenate((X_train_1,X_train_2))
y_train_combined=np.concatenate((y_train_1,y_train_2))

X_val_combined=np.concatenate((X_val_1,X_val_2))
y_val_combined=np.concatenate((y_val_1,y_val_2))

X_test_1=test_data(x_images[0:int(len(x_images)/2)], IMG_SIZE,180)
X_test_2=test_data(x_images[int(len(x_images)/2):], IMG_SIZE,-80)
X_test_combined=np.concatenate((X_test_1,X_test_2))

history_combined,result_combined=retrain_model(settings_trained_models,X_train_combined,X_val_combined,y_train_combined,y_val_combined,X_test_combined,y_test,"combined")
plot_history(history_combined,"combined")
result_combined.to_csv(current_dir+'/Results/CNN_Defense/CNN_10_retrained_combined.csv')